In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from model_assertions.checker import Checker
from model_assertions.consistency import IdentifierConsistencyAssertion, TimeConsistencyAssertion

In [2]:
# In this example, we assume the data is generated from some deep learning method
# We'll assume the dataframe is already populated

df = pd.DataFrame(
    [[0, 1, 'Christi Paul', 'Female', 'Black'],
     [1, 1, 'Christi Paul', 'Female', 'Brown'],
     [3, 1, 'Poppy Harlow', 'Female', 'Black'],
     [4, 1, 'Christi Paul', 'Male', 'Black']],
    columns=['frame', 'scene_idenfier', 'name', 'gender', 'hair_color']
)

In [3]:
df

,frame,scene_idenfier,name,gender,hair_color
0,0,1,Christi Paul,Female,Black
1,1,1,Christi Paul,Female,Brown
2,3,1,Poppy Harlow,Female,Black
3,4,1,Christi Paul,Male,Black


In [4]:
# The prediction function here simply returns the predicted values
# This would normally call a deep learning serving method

def prediction_function(df):
    return df[['name', 'gender', 'hair_color']]

In [5]:
# Define the assertions and register them

gender_consistency = IdentifierConsistencyAssertion('name', 'gender')
hair_color_consistency = IdentifierConsistencyAssertion('scene_idenfier', 'hair_color')
time_consistency = TimeConsistencyAssertion('scene_idenfier', 'frame')

checker = Checker(name='Consistency checker', verbose=False)
checker.register_assertion(gender_consistency.get_assertion(), gender_consistency.get_name())
checker.register_assertion(hair_color_consistency.get_assertion(), hair_color_consistency.get_name())
checker.register_assertion(time_consistency.get_assertion(), time_consistency.get_name())

In [6]:
# Run the prediction and check for errors

def styler(x):
    cm = sns.color_palette('husl', 2)
    colors = []
    for color in cm:
        color = (np.array(color) * 255).astype(int)
        colors.append('background-color:' + f'rgb({color[0]},{color[1]},{color[2]})')
        
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[0, 3] = colors[0]
    df1.iloc[1, 3] = colors[0]
    df1.iloc[2, 4] = colors[0]
    df1.iloc[3, 4] = colors[0]
    df1.iloc[4, 0] = colors[1]
    df1.iloc[5, 0] = colors[1]
    return df1

pred_fn = checker.wrap(prediction_function)
outs = pred_fn(df)
df_err = checker.retrieve_errors()
df_err.style.apply(styler, axis=None)

,frame,scene_idenfier,name,gender,hair_color,assertion,err_idx
0,0,1,Christi Paul,Female,Black,"Identifier consistency `name`, `gender`",0
1,4,1,Christi Paul,Male,Black,"Identifier consistency `name`, `gender`",0
2,0,1,Christi Paul,Female,Black,"Identifier consistency `scene_idenfier`, `hair_color`",1
3,1,1,Christi Paul,Female,Brown,"Identifier consistency `scene_idenfier`, `hair_color`",1
4,1,1,Christi Paul,Female,Brown,"Time consistency `scene_idenfier` over `frame`, window of 3",2
5,3,1,Poppy Harlow,Female,Black,"Time consistency `scene_idenfier` over `frame`, window of 3",2


In [7]:
# Alternate coloring

def styler(column):
    if column.name != 'assertion':
        return [''] * len(column)
    
    uniques = column.unique()
    cm = sns.color_palette('husl', len(uniques))
    cmap = {}
    for unique, color in zip(uniques, cm):
        color = (np.array(color) * 255).astype(int)
        color_text = f'rgb({color[0]},{color[1]},{color[2]})'
        cmap[unique] = color_text

    bg_color = column.map(cmap).fillna('')
    return 'background-color:' + bg_color

# df_err.style.apply(styler)